In [ ]:
%matplotlib widget

In [ ]:
from pathlib import Path

import uproot
import numpy as np
import pandas as pd
import awkward as ak
import matplotlib.pyplot as plt

In [ ]:
data_path = Path("/Users/andreas/Downloads/run4_athena_vs_acts_mu1GeV")

athena_trks = uproot.open(data_path / "trk_athena.root")
acts_trks = uproot.open(data_path / "trk_acts.root")

print(athena_trks["TRKTree"].keys())

truth_columns = [
    "event",

    "truth_n",
    "truth_charge",
    "truth_pt",
    "truth_eta",
    "truth_theta",
    "truth_phi",
    "truth_d0",
    "truth_z0",
    "truth_qOverP",
]

reco_columns = [
    "event",

    "track_n",
    "track_charge",
    "track_pt",
    "track_eta",
    "track_theta",
    "track_phi",
    "track_d0",
    "track_z0",
    "track_qOverP",

    "track_pixHits",
    "track_SCTHits",
    "track_pixHoles",
    "track_SCTHoles",
]

truth_trks = ak.to_dataframe(athena_trks["TRKTree"].arrays(truth_columns, library="ak"), how="outer")
athena_trks = ak.to_dataframe(athena_trks["TRKTree"].arrays(reco_columns, library="ak"), how="outer")
acts_trks = ak.to_dataframe(acts_trks["TRKTree"].arrays(reco_columns, library="ak"), how="outer")

In [ ]:
athena_acts_trks = pd.merge(
    athena_trks[athena_trks["track_n"] == 1].set_index("event").add_prefix("athena_"),
    acts_trks[acts_trks["track_n"] == 1].set_index("event").add_prefix("acts_"),
    how="outer",
    left_index=True,
    right_index=True,
)
truth_athena_acts_trks = pd.merge(
    truth_trks.set_index("event"),
    athena_acts_trks,
    how="outer",
    left_index=True,
    right_index=True,
)
truth_athena_acts_trks

In [ ]:
acts_trks[acts_trks["event"] == 71069]

In [ ]:
diff_mask = ~truth_athena_acts_trks["athena_track_pt"].isna() & truth_athena_acts_trks["acts_track_pt"].isna()
diff = truth_athena_acts_trks[diff_mask][["truth_eta", "truth_phi", "truth_theta", "athena_track_eta", "athena_track_pixHoles", "athena_track_SCTHoles", "acts_track_eta", "acts_track_pixHoles"]]

print(diff)
print(diff.index.values)
print(truth_trks[truth_trks["event"].isin(diff.index.values)].reset_index(names=["nr", None])[["nr", "event"]].values)
print(truth_trks[truth_trks["event"].isin(diff.index.values)].reset_index(names=["nr", None])["nr"].values)

In [ ]:
bins = np.linspace(-0.4, 0.4, 100)
fig, ax = plt.subplots()
ax.hist(athena_trks["track_d0"], bins=bins, histtype="step", label="athena")
ax.hist(acts_trks["track_d0"], bins=bins, histtype="step", label="acts")
ax.legend();

In [ ]:
bins = np.linspace(-150, 150, 100)
fig, ax = plt.subplots()
ax.hist(athena_trks["track_z0"], bins=bins, histtype="step", label="athena")
ax.hist(acts_trks["track_z0"], bins=bins, histtype="step", label="acts")
ax.legend();

In [ ]:
bins = np.linspace(-4, 4, 20)
fig, ax = plt.subplots()
ax.hist(athena_trks["track_eta"], bins=bins, histtype="step", label="athena")
ax.hist(acts_trks["track_eta"], bins=bins, histtype="step", label="acts")
ax.hist(truth_trks["truth_eta"], bins=bins, histtype="step", label="truth")
ax.legend();

In [ ]:
bins = np.linspace(0.6, 1.4, 20)
fig, ax = plt.subplots()
ax.hist(athena_trks[athena_trks["track_eta"] < 2]["track_pt"], bins=bins, histtype="step", label="athena")
ax.hist(acts_trks[acts_trks["track_eta"] < 2]["track_pt"], bins=bins, histtype="step", label="acts")
ax.hist(truth_trks[truth_trks["truth_eta"] < 2]["truth_pt"], bins=bins, histtype="step", label="truth")
ax.legend();